In [17]:
import pandas as pd
import numpy as np
from sklearn.cluster import SpectralClustering
from communities.algorithms import louvain_method, girvan_newman

## 相似度矩阵读取

In [18]:
adj = pd.read_excel("软件/DSM1.xlsx", header=None)
adj_matrix = adj.values
# adj_matrix = ad = [[1,1,0,0,0], [1,1,1,1,0], [0,1,1,0,0], [0,1,0,1,1],[0,0,0,1,1]]

## 规范化拉普拉斯矩阵

In [19]:
d = np.sum(adj_matrix, axis=1).tolist()
d = [i**(-0.5) for i in d]
d_ = np.diag(d)
an = len(adj_matrix)
adj_mat_lap = np.eye(an) - np.matmul(np.matmul(d_,adj_matrix), d_)
print(adj_mat_lap)

[[ 0.58847737 -0.28242447 -0.05016077 -0.06317389 -0.03381003  0.
   0.          0.         -0.05036649 -0.01763002 -0.01966645]
 [-0.28242447  0.66442953 -0.04529593 -0.20805369 -0.03053097 -0.03295434
   0.          0.         -0.0454817  -0.01592018 -0.0177591 ]
 [-0.05016077 -0.04529593  0.72826087 -0.22345994 -0.24726768 -0.14530914
   0.          0.         -0.04092804 -0.01432624 -0.01598105]
 [-0.06317389 -0.20805369 -0.22345994  0.66442953  0.          0.
   0.          0.         -0.0454817  -0.04776053 -0.05327731]
 [-0.03381003 -0.03053097 -0.24726768  0.          0.72222222 -0.23686266
  -0.15469849 -0.0205777  -0.02758686  0.          0.        ]
 [ 0.         -0.03295434 -0.14530914  0.         -0.23686266  0.6763754
  -0.16697755 -0.02221104 -0.02977655  0.          0.        ]
 [ 0.          0.          0.          0.         -0.15469849 -0.16697755
   0.72527473 -0.22101469 -0.13443085 -0.14116642  0.        ]
 [ 0.          0.          0.          0.         -0.02057

## 本证间隙法求最优聚类数

In [20]:
mat, _ = np.linalg.eig(adj_mat_lap)
mat = sorted(mat, reverse=True)

In [21]:
g = [mat[i] - mat[i+1] for i in range(len(mat)-1)]
print(g)
k = 0
for i in range(1, len(g)-1):
    if g[i]>g[i-1] and g[i]>g[i+1]:
        k=i+1
        print(k)
        break

[0.1002415567717807, 0.03779555439898996, 0.031432912916876665, 0.06950615762017998, 0.06672017020741683, 0.17611000336445104, 0.21426401251349358, 0.12053256283485964, 0.046792454710210735, 0.23840287060104728]
4


## 谱聚类求解

In [22]:
sc = SpectralClustering(k, affinity='precomputed')
sc.fit(adj_matrix)
print('spectral clustering')
print(sc.labels_)

spectral clustering
[2 2 3 2 3 3 1 1 0 0 0]


In [23]:
def get_q(adj_m, c_, c_d):
    in_ = 0
    tot_ = 0
    in_list = []
    tot_list = []
    for i in c_d.keys():
        if i == c_:
            in_list.extend(c_d[i])
        tot_list.extend(c_d[i])
    for i in in_list:
        for z in in_list:
                in_ += adj_m[i][z]
    for i in in_list:
        for z in tot_list:
            tot_ += adj_m[i][z]
    return in_, tot_


In [24]:
def get_Q(adj_mat, sc_, k_):
    m = 0
    adj_matrix_ = adj_mat - np.eye(len(adj_mat))
    for i in range(len(adj_matrix_)):
        for j in range(len(adj_matrix_[0])):
            m += adj_matrix_[i][j]
    m/=2
    c_list = sc_.labels_.tolist()
    c_dict = {}
    for i in range(len(c_list)):
        c_dict.setdefault(c_list[i],[])
        c_dict[c_list[i]].append(i)
    Q = 0
    for i in c_dict.keys():
        n,t = get_q(adj_matrix_, i, c_dict)
        Q += n/(2*m) - (t/(2*m))**2
    print(Q)

In [25]:
print(adj_matrix)
for o in range(1, 10):
    sc = SpectralClustering(o, affinity='precomputed', n_init=10)
    sc.fit(adj_matrix)
    community = sc.labels_.tolist()
    com_dict = {c: [i for i, val in enumerate(community) if val == c] for c in set(community)}
    comm = [set(val) for val in com_dict.values()]
    print(comm, end="\t")
    get_Q(adj_matrix, sc, o)

[[1.   0.76 0.15 0.17 0.1  0.   0.   0.   0.15 0.05 0.05]
 [0.76 1.   0.15 0.62 0.1  0.1  0.   0.   0.15 0.05 0.05]
 [0.15 0.15 1.   0.74 0.9  0.49 0.   0.   0.15 0.05 0.05]
 [0.17 0.62 0.74 1.   0.   0.   0.   0.   0.15 0.15 0.15]
 [0.1  0.1  0.9  0.   1.   0.79 0.56 0.05 0.1  0.   0.  ]
 [0.   0.1  0.49 0.   0.79 1.   0.56 0.05 0.1  0.   0.  ]
 [0.   0.   0.   0.   0.56 0.56 1.   0.54 0.49 0.49 0.  ]
 [0.   0.   0.   0.   0.05 0.05 0.54 1.   0.   0.   0.  ]
 [0.15 0.15 0.15 0.15 0.1  0.1  0.49 0.   1.   0.76 0.6 ]
 [0.05 0.05 0.05 0.15 0.   0.   0.49 0.   0.76 1.   0.76]
 [0.05 0.05 0.05 0.15 0.   0.   0.   0.   0.6  0.76 1.  ]]
[{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}]	0.0
[{0, 1, 2, 3, 8, 9, 10}, {4, 5, 6, 7}]	0.20271693533246754
[{8, 9, 10}, {0, 1, 2, 3}, {4, 5, 6, 7}]	0.30488887884838145
[{6, 7}, {0, 1, 3}, {8, 9, 10}, {2, 4, 5}]	0.2953265939024173
[{4, 5, 6}, {7}, {8, 9, 10}, {2, 3}, {0, 1}]	0.23236352418693315
[{4, 5}, {9, 10}, {0, 1}, {7}, {2, 3}, {8, 6}]	0.11867827799412474
[{0, 1

In [26]:
adj_matrix = adj_matrix - np.eye(len(adj_matrix))
print(adj_matrix)

[[0.   0.76 0.15 0.17 0.1  0.   0.   0.   0.15 0.05 0.05]
 [0.76 0.   0.15 0.62 0.1  0.1  0.   0.   0.15 0.05 0.05]
 [0.15 0.15 0.   0.74 0.9  0.49 0.   0.   0.15 0.05 0.05]
 [0.17 0.62 0.74 0.   0.   0.   0.   0.   0.15 0.15 0.15]
 [0.1  0.1  0.9  0.   0.   0.79 0.56 0.05 0.1  0.   0.  ]
 [0.   0.1  0.49 0.   0.79 0.   0.56 0.05 0.1  0.   0.  ]
 [0.   0.   0.   0.   0.56 0.56 0.   0.54 0.49 0.49 0.  ]
 [0.   0.   0.   0.   0.05 0.05 0.54 0.   0.   0.   0.  ]
 [0.15 0.15 0.15 0.15 0.1  0.1  0.49 0.   0.   0.76 0.6 ]
 [0.05 0.05 0.05 0.15 0.   0.   0.49 0.   0.76 0.   0.76]
 [0.05 0.05 0.05 0.15 0.   0.   0.   0.   0.6  0.76 0.  ]]


In [27]:
c, _ = louvain_method(adj_matrix)
print(c)

[{0, 1, 3}, {2, 4, 5, 6, 7}, {8, 9, 10}]


In [28]:
c, _ = girvan_newman(adj_matrix)
print(c)

[{0, 9, 10, 3}, {1, 2, 4, 5, 6, 7, 8}]
